# Load Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv('areaBasedList.csv')

In [3]:
data.head(2)

,addr1,areacode,cat1,cat2,cat3,contentid,contenttypeid,createdtime,firstimage,firstimage2,mapx,mapy,mlevel,modifiedtime,readcount,sigungucode,tel,title,zipcode
0,경기도 고양시 일산동구 호수로 595,31.0,A02,A0207,A02070200,141799,15,20070424000000,http://tong.visitkorea.or.kr/cms/resource/80/2...,http://tong.visitkorea.or.kr/cms/resource/80/2...,126.768685,37.653974,6.0,20200325104012,572045.0,2.0,031-908-7750,고양국제꽃박람회 2020,10400
1,제주특별자치도 서귀포시 남원읍 신례동로 256,39.0,A02,A0202,A02020600,322836,12,20071211113035,http://tong.visitkorea.or.kr/cms/resource/47/2...,http://tong.visitkorea.or.kr/cms/resource/47/2...,126.634432,33.308517,6.0,20200526141048,499370.0,3.0,NaN,휴애리자연생활공원,63608


# TODO: Crawl overview data and establish lda model to be contained in content-filtering

import requests
from bs4 import BeautifulSoup

serviceKey = "brK8velu2xJHc6C0XfwxtDK10X2i9gc0aovlw5vEpL18%2FNx9YXjg5eagU5MG7h6VqG5R0%2BxNyYPyze2cl5yG6Q%3D%3D"

url = "http://api.visitkorea.or.kr/openapi/service/rest/KorWithService/detailCommon?"

queryParams = "?" + "&numOfRows=&pageNo=&MobileOS=ETC&MobileApp=AppTest&" + \
    "&contentId=" + contentId +"&contentTypeId="+ contentTypeId + \
    "&defaultYN=&firstImageYN=&areacodeYN=&catcodeYN=&addrinfoYN=&mapinfoYN=&overviewYN=Y&"

request = Request(url + queryParams)
request.get_method = lambda: 'GET'
response_body = urlopen(request)



In [5]:
data['category'] = data['cat1']+" "+data['cat2']+" "+data['cat3']

In [6]:
data.head(2)

,addr1,areacode,cat1,cat2,cat3,contentid,contenttypeid,createdtime,firstimage,firstimage2,mapx,mapy,mlevel,modifiedtime,readcount,sigungucode,tel,title,zipcode,category
0,경기도 고양시 일산동구 호수로 595,31.0,A02,A0207,A02070200,141799,15,20070424000000,http://tong.visitkorea.or.kr/cms/resource/80/2...,http://tong.visitkorea.or.kr/cms/resource/80/2...,126.768685,37.653974,6.0,20200325104012,572045.0,2.0,031-908-7750,고양국제꽃박람회 2020,10400,A02 A0207 A02070200
1,제주특별자치도 서귀포시 남원읍 신례동로 256,39.0,A02,A0202,A02020600,322836,12,20071211113035,http://tong.visitkorea.or.kr/cms/resource/47/2...,http://tong.visitkorea.or.kr/cms/resource/47/2...,126.634432,33.308517,6.0,20200526141048,499370.0,3.0,NaN,휴애리자연생활공원,63608,A02 A0202 A02020600


In [7]:
data.category.head(2)

0    A02 A0207 A02070200
1    A02 A0202 A02020600
Name: category, dtype: object

In [17]:
data.category = data.category.fillna(" ")

In [22]:
data.category.isnull().sum()

0

In [23]:
count_vector = CountVectorizer(ngram_range=(1, 3))

In [24]:
c_vector_category = count_vector.fit_transform(data['category'])

In [25]:
c_vector_category.shape

(5841, 399)

In [26]:
cat_c_sim = cosine_similarity(c_vector_category, c_vector_category).argsort()[:, ::-1]

In [27]:
cat_c_sim.shape

(5841, 5841)

In [28]:
def get_recommend_place_list_content(df, contentid, top=30):
    # 특정 장소와 비슷한 장소를 추천해야 하기 때문에 '특정 장소' 정보를 뽑아낸다.
    target_place_index = df[df['contentid'] == contentid].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = cat_c_sim[target_place_index, :top].reshape(-1)
    #본인을 제외
    sim_index = sim_index[sim_index != target_place_index]

    #data frame으로 만들고 readcount순으로 정렬한 뒤 return
    result = df.iloc[sim_index].sort_values('readcount', ascending=False)[:10]
    return result

In [32]:
get_recommend_place_list_content(data, contentid=322836)

,addr1,areacode,cat1,cat2,cat3,contentid,contenttypeid,createdtime,firstimage,firstimage2,mapx,mapy,mlevel,modifiedtime,readcount,sigungucode,tel,title,zipcode,category
273,서울특별시 송파구 올림픽로 240,1.0,A02,A0202,A02020600,126498,12,20031105000000,http://tong.visitkorea.or.kr/cms/resource/77/2...,http://tong.visitkorea.or.kr/cms/resource/77/2...,127.097901,37.511352,6.0,20200603092113,69227.0,18.0,NaN,롯데월드,05554,A02 A0202 A02020600
286,대전광역시 중구 사정공원로 70,3.0,A02,A0202,A02020600,127641,12,20031106000000,http://tong.visitkorea.or.kr/cms/resource/57/2...,http://tong.visitkorea.or.kr/cms/resource/57/2...,127.400129,36.287312,6.0,20200608161115,67779.0,5.0,NaN,대전오월드,35073,A02 A0202 A02020600
302,대전광역시 유성구 대덕대로 480,3.0,A02,A0202,A02020600,125994,12,20060616000000,http://tong.visitkorea.or.kr/cms/resource/29/2...,http://tong.visitkorea.or.kr/cms/resource/29/2...,127.384990,36.377546,6.0,20200608163328,66024.0,4.0,NaN,대전엑스포과학공원,34126,A02 A0202 A02020600
327,경기도 용인시 처인구 포곡읍 에버랜드로 199,31.0,A02,A0202,A02020600,127797,12,20030827000000,http://tong.visitkorea.or.kr/cms/resource/56/2...,http://tong.visitkorea.or.kr/cms/resource/56/2...,127.201339,37.293379,6.0,20200515113342,63925.0,23.0,NaN,에버랜드,17023,A02 A0202 A02020600
698,전라남도 순천시 국가정원1호길 47,38.0,A02,A0202,A02020600,1919548,12,20140516145011,http://tong.visitkorea.or.kr/cms/resource/16/1...,http://tong.visitkorea.or.kr/cms/resource/16/1...,127.508239,34.930015,6.0,20200513133314,46624.0,11.0,NaN,순천만 국가정원,58026,A02 A0202 A02020600
717,경기도 과천시 광명로 181,31.0,A02,A0202,A02020600,126719,12,20031113000000,http://tong.visitkorea.or.kr/cms/resource/53/2...,http://tong.visitkorea.or.kr/cms/resource/53/2...,127.018899,37.433687,6.0,20200518154357,46049.0,3.0,NaN,서울랜드,13829,A02 A0202 A02020600
952,전라남도 해남군 문내면 관광레저로 12,38.0,A02,A0202,A02020600,126313,12,20031029000000,http://tong.visitkorea.or.kr/cms/resource/48/2...,http://tong.visitkorea.or.kr/cms/resource/48/2...,126.310967,34.572296,6.0,20200116144422,41165.0,23.0,NaN,우수영관광지,59004,A02 A0202 A02020600
996,경기도 김포시 하성면 하성로 585,31.0,A02,A0202,A02020600,128156,12,20040531000000,http://tong.visitkorea.or.kr/cms/resource/13/1...,http://tong.visitkorea.or.kr/cms/resource/13/1...,126.627604,37.725199,6.0,20191223165840,40477.0,8.0,NaN,태산패밀리파크,10008,A02 A0202 A02020600
1247,인천광역시 중구 월미문화로 81,2.0,A02,A0202,A02020600,949266,12,20100211093321,http://tong.visitkorea.or.kr/cms/resource/58/1...,http://tong.visitkorea.or.kr/cms/resource/58/1...,126.596047,37.471483,6.0,20200110111918,36722.0,10.0,NaN,월미테마파크,22303,A02 A0202 A02020600
1646,경상남도 양산시 하북면 통도7길 68,36.0,A02,A0202,A02020600,126926,12,20040203000000,http://tong.visitkorea.or.kr/cms/resource/82/2...,http://tong.visitkorea.or.kr/cms/resource/82/2...,129.079142,35.498118,6.0,20200130134750,32064.0,10.0,NaN,통도환타지아,50502,A02 A0202 A02020600
